In [ ]:
import pandas as pd

In [ ]:
# Populating with dummy data
df = pd.DataFrame({
    "ID": list(range(5)),
    "Type": ["Expense", "Expense", "Expense", "Reimbursement", "Income"],
    "Amount": [300, 14, 25, 500, 100]
})

In [164]:
df_transactions = df
df_transactions

,ID,Type,Amount
0,0,Expense,300
1,1,Expense,14
2,2,Expense,25
3,3,Reimbursement,500
4,4,Income,100


In [ ]:
# Selecting reimbursements
df_reimbursements = df[df["Type"] == "Reimbursement"]
df_reimbursements

,ID,Type,Amount
3,3,Reimbursement,500


In [ ]:
# Composing a temporary table to specify what part of reimbursement goes for which expense
df_reimbursements_decomposed = pd.DataFrame(columns=["Reimbursement_ID", "Expense_ID", "Amount"])
df_reimbursements_decomposed.loc[len(df_reimbursements_decomposed)] = [3, 0, 300] # Selected reimbursement from original transactions + reimbursement partial sum from user
df_reimbursements_decomposed.loc[len(df_reimbursements_decomposed)] = [3, 1, 13] # Selected reimbursement from original transactions + reimbursement partial sum from user
df_reimbursements_decomposed.loc[len(df_reimbursements_decomposed)] = [3, 2, 25] # Selected reimbursement from original transactions + reimbursement partial sum from user
df_reimbursements_decomposed

,Reimbursement_ID,Expense_ID,Amount
0,3,0,300
1,3,1,13
2,3,2,25


In [ ]:
# Step that handles the reimbursement amount
df_transactions_reimbursed = df_transactions.merge(df_reimbursements_decomposed[["Expense_ID", "Amount"]], how="left", left_on="ID", right_on="Expense_ID")
df_transactions_reimbursed["Amount"] = df_transactions_reimbursed.apply(lambda row: row["Amount_x"] - row["Amount_y"] if pd.notna(row["Amount_y"]) else row["Amount_x"], axis=1)
df_transactions_reimbursed = df_transactions_reimbursed.drop(["Amount_x", "Amount_y"], axis=1)
df_transactions_reimbursed

,ID,Type,Expense_ID,Amount
0,0,Expense,0.0,0.0
1,1,Expense,1.0,1.0
2,2,Expense,2.0,0.0
3,3,Reimbursement,NaN,500.0
4,4,Income,NaN,100.0


In [ ]:
# Step that resolves the "Reimbursement" transaction:
# - A - Not all sum used --> the transaction type is changed to "Income"
# - B - All sum used --> the transaction is deleted
total_reimbursed = df_reimbursements_decomposed["Amount"].sum()
if total_reimbursed < df_transactions_reimbursed[df_transactions_reimbursed["ID"] == 3]["Amount"].sum():
    df_transactions_reimbursed.loc[df_transactions_reimbursed["ID"] == 3, "Type"] = "Income" 
    df_transactions_reimbursed.loc[df_transactions_reimbursed["ID"] == 3, "Amount"] -= total_reimbursed
else:
    df_transactions_reimbursed.drop(df_transactions_reimbursed["ID"] == 3, inplace=True)
df_transactions_reimbursed

,ID,Type,Expense_ID,Amount
0,0,Expense,0.0,0.0
1,1,Expense,1.0,1.0
2,2,Expense,2.0,0.0
3,3,Income,NaN,162.0
4,4,Income,NaN,100.0
